# Huffman Coding

In [1]:
#include <iostream>
#include <string>
#include <queue>       // std::priority_queue
#include <vector>
#include <functional>  // std::greater
#include <unordered_map>


class HeapNode
{
    HeapNode* left = nullptr;
    HeapNode* right = nullptr;
    double freq;
    char ch;

public: 
    HeapNode(const double& freq, const char& ch, HeapNode* left=nullptr, HeapNode* right=nullptr) 
        : freq(freq), ch(ch), left(left), right(right) {}
    
    void print() const { std::cout << "(" << ch << ", " << freq << ")" << std::endl; }
    
    friend bool operator<(const HeapNode& a, const HeapNode& b) { 
        return a.freq < b.freq; 
    }
};

std::string message(R"(In computer science and information theory, a Huffman code is a particular type of optimal prefix code that is commonly used for lossless data compression. The process of finding or using such a code proceeds by means of Huffman coding, an algorithm developed by David A. Huffman while he was a Sc.D. student at MIT, and published in the 1952 paper "A Method for the Construction of Minimum-Redundancy Codes".)");

std::unordered_map<char, int> freq;
for (const char& c : message)
    freq[c]++;

std::vector<HeapNode> nodes;
for (auto [letter, frequency] : freq)
    nodes.push_back(HeapNode(frequency, letter));

std::priority_queue<HeapNode> q(std::less<HeapNode>(), nodes);

while(!q.empty()) 
{
    const auto& p = q.top();
    p.print();
    q.pop();
}


input_line_8:25:31: error: no matching constructor for initialization of 'std::priority_queue<HeapNode>'
std::priority_queue<HeapNode> q(std::greater<HeapNode>(), nodes);
                              ^ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/usr/lib/gcc/x86_64-linux-gnu/7.5.0/../../../../include/c++/7.5.0/bits/stl_queue.h:477:7: note: candidate constructor not viable: no known conversion from 'std::greater<HeapNode>' to 'const std::less<__cling_N52::HeapNode>' for 1st argument
      priority_queue(const _Compare& __x, const _Sequence& __s)
      ^
/usr/lib/gcc/x86_64-linux-gnu/7.5.0/../../../../include/c++/7.5.0/bits/stl_queue.h:482:7: note: candidate constructor not viable: no known conversion from 'std::greater<HeapNode>' to 'const std::less<__cling_N52::HeapNode>' for 1st argument
      priority_queue(const _Compare& __x, _Sequence&& __s = _Sequence())
      ^
/usr/lib/gcc/x86_64-linux-gnu/7.5.0/../../../../include/c++/7.5.0/bits/stl_queue.h:552:2: note: candidate template ignored: deduce

Interpreter Error: 